In [36]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

### 기업 정보 버튼 누르기

In [37]:
prefix = "https://www.saramin.co.kr/zf_user/search?keydownAccess=&searchword="
searchword = "로봇 SW 개발"
url = prefix + searchword.replace(" ", "+")
driver = webdriver.Chrome()
driver.set_window_size(1400, 1080)

for _ in range(2):
    driver.get(url)
    time.sleep(1)
company_link = driver.find_element(By.CSS_SELECTOR, ".tab_search_result.on li a[target='company']")
company_link.click()

### 기업 링크 리스트 추출

In [ ]:
href_list = [comany_link.get_attribute("href") for comany_link in driver.find_elements(By.CSS_SELECTOR, ".corp_name a")]
href_list

### URL 타고 넘어가서 기업 정보 추출

In [ ]:
driver.get(href_list[1])
title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
key_info = driver.find_elements(By.CLASS_NAME, "company_summary_desc")
for info in key_info:
    print(info.text)


In [ ]:
driver.get(href_list[0])
title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
key_info = driver.find_elements(By.CLASS_NAME, "company_summary_desc")
company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
company_details = driver.find_elements(By.CLASS_NAME, "company_details_group dd")
# 기본 정보 설정
company_info = {"기업명": title}

for i, form_element in enumerate(company_form):
    if i == 0 and " " in form_element.text:  # 첫 번째 요소가 "업력 10년" 같은 형식인 경우
        parts = form_element.text.split(" ", 1)
        if len(parts) >= 2:
            company_info[parts[0]] = parts[1]
        else:
            company_info[f"정보{i+1}"] = form_element.text
    elif i < len(key_info):  # key_info에 해당하는 인덱스가 있는 경우
        company_info[key_info[i].text] = form_element.text
    else:  # key_info에 해당하는 인덱스가 없는 경우
        company_info[f"정보{i+1}"] = form_element.text
company_info

### 연봉 정보(최소, 평균, 최대)

In [ ]:
menu_ul_element = driver.find_element(By.CLASS_NAME, "menu_list")
li_list = menu_ul_element.find_elements(By.TAG_NAME, "li")
for li in li_list:
    if li.text == "연봉정보":
        li.click()
        break
avg_salary = driver.find_element(By.CLASS_NAME, "average_currency").text
min_salary = driver.find_element(By.CLASS_NAME, "min_txt").text
max_salary = driver.find_element(By.CLASS_NAME, "max_txt").text
company_info["연봉정보"] = {"최소" : min_salary, "평균" : avg_salary, "최대" : max_salary}
company_info

### 셀 하나로 병합

In [ ]:
company_info_list = []
for href in href_list:
    driver.get(href)
    time.sleep(1)
    
    # 기본 정보 수집
    title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
    company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
    key_info = driver.find_elements(By.CLASS_NAME, "company_summary_desc")
    company_info = {"기업명": title}

    # ... existing code for company info ...



    for i, form_element in enumerate(company_form):
        if i == 0 and " " in form_element.text:  # 첫 번째 요소가 "업력 10년" 같은 형식인 경우
            parts = form_element.text.split(" ", 1)
            if len(parts) >= 2:
                company_info[parts[0]] = parts[1]
            else:
                company_info[f"정보{i+1}"] = form_element.text
        elif i < len(key_info):  # key_info에 해당하는 인덱스가 있는 경우
            company_info[key_info[i].text] = form_element.text
        else:  # key_info에 해당하는 인덱스가 없는 경우
            company_info[f"정보{i+1}"] = form_element.text
    company_info
    # 연봉 정보 수집
    try:
        menu_ul_element = driver.find_element(By.CLASS_NAME, "menu_list")
        li_list = menu_ul_element.find_elements(By.TAG_NAME, "li")
        salary_found = False
        
        for li in li_list:
            if li.text == "연봉정보":
                li.click()
                time.sleep(2)  # 페이지 로딩을 위한 대기 시간 증가
                
                try:
                    # 연봉 정보가 있는지 확인
                    if len(driver.find_elements(By.CLASS_NAME, "average_currency")) > 0:
                        avg_salary = driver.find_element(By.CLASS_NAME, "average_currency").text
                        min_salary = driver.find_element(By.CLASS_NAME, "min_txt").text
                        max_salary = driver.find_element(By.CLASS_NAME, "max_txt").text
                        company_info["연봉정보"] = {"최소": min_salary, "평균": avg_salary, "최대": max_salary}
                        salary_found = True
                    else:
                        company_info["연봉정보"] = "None"
                except Exception as e:
                    print(f"연봉 정보 추출 중 오류: {str(e)}")
                    company_info["연봉정보"] = "None"
                break
        
        if not salary_found:
            company_info["연봉정보"] = "None"
            
    except Exception as e:
        print(f"연봉 정보 페이지 접근 중 오류: {str(e)}")
        company_info["연봉정보"] = "None"
    
    company_info_list.append(company_info)

In [ ]:
company_info_list

### 반복문으로 써주기 위해 함수화

In [ ]:
def collect_company_info(href_list, driver):
    company_info_list = []
    for href in href_list:
        driver.get(href)
        time.sleep(1)
        
        # 기본 정보 수집
        title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
        company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
        key_info = driver.find_elements(By.CLASS_NAME, "company_summary_desc")
        company_info = {"기업명": title}

        for i, form_element in enumerate(company_form):
            if i == 0 and " " in form_element.text:  # 첫 번째 요소가 "업력 10년" 같은 형식인 경우
                parts = form_element.text.split(" ", 1)
                if len(parts) >= 2:
                    company_info[parts[0]] = parts[1]
                else:
                    company_info[f"정보{i+1}"] = form_element.text
            elif i < len(key_info):  # key_info에 해당하는 인덱스가 있는 경우
                company_info[key_info[i].text] = form_element.text
            else:  # key_info에 해당하는 인덱스가 없는 경우
                company_info[f"정보{i+1}"] = form_element.text

        # 연봉 정보 수집
        try:
            menu_ul_element = driver.find_element(By.CLASS_NAME, "menu_list")
            li_list = menu_ul_element.find_elements(By.TAG_NAME, "li")
            salary_found = False
            
            for li in li_list:
                if li.text == "연봉정보":
                    li.click()
                    time.sleep(2)
                    
                    try:
                        if len(driver.find_elements(By.CLASS_NAME, "average_currency")) > 0:
                            avg_salary = driver.find_element(By.CLASS_NAME, "average_currency").text
                            min_salary = driver.find_element(By.CLASS_NAME, "min_txt").text
                            max_salary = driver.find_element(By.CLASS_NAME, "max_txt").text
                            company_info["연봉정보"] = {"최소": min_salary, "평균": avg_salary, "최대": max_salary}
                            salary_found = True
                        else:
                            company_info["연봉정보"] = "None"
                    except Exception as e:
                        print(f"연봉 정보 추출 중 오류: {str(e)}")
                        company_info["연봉정보"] = "None"
                    break
            
            if not salary_found:
                company_info["연봉정보"] = "None"
                
        except Exception as e:
            print(f"연봉 정보 페이지 접근 중 오류: {str(e)}")
            company_info["연봉정보"] = "None"
        
        company_info_list.append(company_info)
    
    return company_info_list


company_info_list = collect_company_info(href_list, driver)
company_info_list

### 다시 원래 페이지로 돌아간 뒤 스크롤 내리기


In [64]:
# 원래 페이지로
driver.execute_script(f"window.history.go(-{len(href_list)})")
time.sleep(1)
content_bottom = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "content_bottom")))
driver.execute_script("arguments[0].scrollIntoView();", content_bottom)
time.sleep(1)


A 태그는 10개이다. 2번부터 다음 페이지 버튼까지

In [ ]:
bottom_ul_list = content_bottom.find_elements(By.TAG_NAME, "a")
len(bottom_ul_list)


### 페이지를 넘기며 반복해서 데이터 수집(전체 쉘 통합)

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
import time
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

def collect_company_info(href_list, driver):
    company_info_list = []
    for href in href_list:
        driver.get(href)
        time.sleep(1)
        
        # 기본 정보 수집
        title = driver.find_element(By.CLASS_NAME, "tit_company").get_attribute("title")
        company_form = driver.find_element(By.CLASS_NAME, "company_summary").find_elements(By.CLASS_NAME, "company_summary_tit")
        key_info = driver.find_elements(By.CLASS_NAME, "company_summary_desc")
        company_info = {"기업명": title}

        for i, form_element in enumerate(company_form):
            if i == 0 and " " in form_element.text:  # 첫 번째 요소가 "업력 10년" 같은 형식인 경우
                parts = form_element.text.split(" ", 1)
                if len(parts) >= 2:
                    company_info[parts[0]] = parts[1]
                else:
                    company_info[f"정보{i+1}"] = form_element.text
            elif i < len(key_info):  # key_info에 해당하는 인덱스가 있는 경우
                company_info[key_info[i].text] = form_element.text
            else:  # key_info에 해당하는 인덱스가 없는 경우
                company_info[f"정보{i+1}"] = form_element.text
        company_details = {}
        detail_groups = driver.find_elements(By.CLASS_NAME, "company_details_group")
        
        for group in detail_groups:
            try:
                title = group.find_element(By.CLASS_NAME, "tit").text
                desc = group.find_element(By.CLASS_NAME, "desc").text
                
                if "..." in desc:
                    desc = group.find_element(By.CLASS_NAME, "ellipsis").get_attribute("title")
                
                company_details[title] = desc
            except:
                continue
        
        # 상세 정보를 company_info에 추가
        company_info["상세정보"] = company_details
        # 연봉 정보 수집
        try:
            menu_ul_element = driver.find_element(By.CLASS_NAME, "menu_list")
            li_list = menu_ul_element.find_elements(By.TAG_NAME, "li")
            salary_found = False
            
            for li in li_list:
                if li.text == "연봉정보":
                    li.click()
                    time.sleep(2)
                    
                    try:
                        if len(driver.find_elements(By.CLASS_NAME, "average_currency")) > 0:
                            avg_salary = driver.find_element(By.CLASS_NAME, "average_currency").text
                            min_salary = driver.find_element(By.CLASS_NAME, "min_txt").text
                            max_salary = driver.find_element(By.CLASS_NAME, "max_txt").text
                            company_info["연봉정보"] = {"최소": min_salary, "평균": avg_salary, "최대": max_salary}
                            salary_found = True
                        else:
                            company_info["연봉정보"] = "None"
                    except Exception as e:
                        print(f"연봉 정보 추출 중 오류: {str(e)}")
                        company_info["연봉정보"] = "None"
                    break
            
            if not salary_found:
                company_info["연봉정보"] = "None"
                
        except Exception as e:
            print(f"연봉 정보 페이지 접근 중 오류: {str(e)}")
            company_info["연봉정보"] = "None"
        
        company_info_list.append(company_info)
    
    return company_info_list

def main():
    company_info_list = []
    prefix = "https://www.saramin.co.kr/zf_user/search?keydownAccess=&searchword="
    searchword = "로봇 SW 개발"
    url = prefix + searchword.replace(" ", "+")
    driver = webdriver.Chrome()
    driver.set_window_size(1400, 1080)
    driver.get(url)
    time.sleep(1)

    # 기업 정보 클릭
    company_link = driver.find_element(By.CSS_SELECTOR, ".tab_search_result.on li a[target='company']")
    company_link.click()

    
    # 페이지 넘기면서 순환
    while True:
        href_list = [comany_link.get_attribute("href") for comany_link in driver.find_elements(By.CSS_SELECTOR, ".corp_name a")]
        current_url = driver.current_url
        print(current_url)
        company_info_list.extend(collect_company_info(href_list, driver))
        # 원래 화면으로 돌아가기
        driver.get(current_url)
        time.sleep(1)
        # 다음 페이지 버튼 클릭
        content_bottom = WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CLASS_NAME, "content_bottom")))
        driver.execute_script("arguments[0].scrollIntoView();", content_bottom)
        current_page = int(content_bottom.find_element(By.CSS_SELECTOR, "span.page").text)
        try :
            next_page_btn = content_bottom.find_element(By.CSS_SELECTOR, f"a[page='{current_page + 1}']")
            next_page_btn.click()
            time.sleep(1)
            if current_page % 10 == 0:
                temp_df = pd.DataFrame(company_info_list)
                temp_df.to_csv("../data/company_info.csv", encoding="utf-8", index=False)
                current_page = driver.current_url
                driver.quit()
                driver = webdriver.Chrome()
                driver.set_window_size(1400, 1080)
                driver.get(current_url)
                time.sleep(1)
        except :
            break
    return company_info_list

if __name__ == "__main__":
    company_info_list = main()
    df = pd.DataFrame(company_info_list)
    df.to_csv("../data/company_info.csv")


In [ ]:
len(df)
df